In [15]:
import pandas as pd 
import numpy as np 
import os
import re
import matplotlib.pyplot as plt
from Bio import SeqIO
from itertools import groupby
import seaborn as sns
currdir = os.getcwd()
parent = os.path.dirname(currdir)
gparent = os.path.dirname(parent)


In [16]:
AnnotatedKinaseSheet = f'../10415/10415_unbiased_normalized_041124_AnnotatedHumanKinome.csv'
PhosphoSheet = f'../10415/10415_phos_normalized_041124.xlsx'

data = pd.read_csv(AnnotatedKinaseSheet, index_col=0)
phospho = pd.read_excel(f'{PhosphoSheet}', sheet_name='Normalized Data')

data['Human_Kinase'] = data['Human_Kinase'].replace('False', False)

kinases = data[data['Human_Kinase'] != False].reset_index(drop=True)

out = list(set(phospho['Accession']) & set(kinases['Accession']))
print(len(out), kinases.shape)

# make non-phospho kinases unique
if '_k' not in kinases.loc[0, 'Accession'].lower():
    kinases['Accession'] = kinases['Accession'].apply(lambda x: f'{x.split(";")[0]}_k')


235 (316, 53)


In [17]:
phospho

,Accession,Description,Genes,Stripped_Seq,Sequence,Detected_Imputed,PeptidePosition,PTMLocations,112744 Fxn10_Control Normalized,112758 Fxn10_Control Normalized,...,Fxn5_Rxn_CV,Fxn6_Control_CV,Fxn6_Rxn_CV,Fxn7_Control_CV,Fxn7_Rxn_CV,Fxn8_Control_CV,Fxn8_Rxn_CV,Fxn9_Control_CV,Fxn9_Rxn_CV,SPQC_CV
0,A0A0B4J2A2,"Peptidyl-prolyl cis-trans isomerase A-like 4C,...",PPIAL4C,HTGSGILSMANAGPNTNGSQFFICTAK,_HTGSGILSMANAGPNT[Phospho (STY)]NGSQFFIC[Carba...,-.-.-.-.-.1.-.-.-.-.-.-.-.-.-.-.-.-.-.1.-.-.-....,92,"(T107,C115)",3.519489,10.235566,...,66.90,48.80,90.7,38.9,58.3,86.9,81.1,119.00,117.0,46.3
1,A0A0B4J2D5;P0DPI2,Putative glutamine amidotransferase-like class...,GATD3B;GATD3,NLSTFAVDGKDCKVNK,_NLS[Phospho (STY)]T[Phospho (STY)]FAVDGKDC[Ca...,-.-.1.-.-.1.-.1.1.1.1.1.1.1.1.1.1.-.-.1.1.-.-....,142;142,"(S144,T145,C153);(S144,T145,C153)",5.129906,3.637086,...,5.53,8.06,54.1,40.6,44.9,26.7,75.2,170.00,12.4,80.6
2,A0A0B4J2F2;P57059,Putative serine/threonine-protein kinase SIK1B...,SIK1B;SIK1,GGLSPFHAPAQSPGLHGGAAGSR,_GGLSPFHAPAQS[Phospho (STY)]PGLHGGAAGSR_,-.-.-.1.1.1.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.1....,623;623,(S634);(S634),8.803518,3.965825,...,86.30,99.30,59.0,53.2,146.0,37.3,67.7,57.90,18.6,131.0
3,A0A0B4J2F2;P57059,Putative serine/threonine-protein kinase SIK1B...,SIK1B;SIK1,GGLSPFHAPAQSPGLHGGAAGSR,_GGLS[Phospho (STY)]PFHAPAQSPGLHGGAAGSR_,-.-.-.1.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-....,623;623,(S626);(S626),8.924532,13.614234,...,53.50,56.50,41.8,91.8,63.4,38.2,34.7,6.79,12.7,45.6
4,A0A0B4J2F2;P57059,Putative serine/threonine-protein kinase SIK1B...,SIK1B;SIK1,PRPVSPSSLLDTAISEEAR,_PRPVSPS[Phospho (STY)]SLLDTAISEEAR_,1.-.-.-.-.-.1.1.-.-.-.1.-.1.-.-.-.1.-.-.-.1.-....,431;431,(S437);(S437),1.545432,1.601255,...,69.60,45.60,18.6,56.8,68.6,47.6,69.5,49.10,45.4,77.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39167,Q9Y6Y8,"SEC23-interacting protein, org=Homo sapiens",SEC23IP,VVESPDFSKDEDYLGK,_VVES[Phospho (STY)]PDFSKDEDYLGK_,1.1.-.1.1.2.1.1.-.1.1.1.1.2.-.1.2.-.2.1.1.2.-....,923,(S926),90.819400,114.393484,...,23.90,27.40,24.4,39.9,36.7,49.3,18.9,34.30,22.3,25.1
39168,TRYP_PIG,"Trypsin, org=Sus scrofa",NaN,LSSPATLNSR,_LSSPATLNS[Phospho (STY)]R_,1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1....,98,(S106),7.512516,5.905145,...,46.90,32.40,31.2,34.9,85.8,81.0,30.3,161.00,152.0,78.2
39169,TRYP_PIG,"Trypsin, org=Sus scrofa",NaN,LSSPATLNSR,_LSSPAT[Phospho (STY)]LNSR_,-.1.-.-.1.-.1.1.-.1.1.1.-.1.1.-.1.1.-.1.1.1.1....,98,(T103),624.474396,896.785816,...,40.60,45.10,66.9,87.8,54.2,31.6,97.7,47.30,85.7,46.2
39170,TRYP_PIG,"Trypsin, org=Sus scrofa",NaN,VATVSLPR,_VATVS[Phospho (STY)]LPR_,-.1.-.-.1.-.1.-.-.1.-.-.-.1.-.1.1.1.1.1.1.-.-....,108,(S112),9.375021,6.000127,...,48.50,29.60,29.4,50.5,32.2,50.6,24.2,47.80,61.1,62.0


In [18]:
def keep_phospho(annotatedseq):
    matches = re.findall(r"\[(.*?)\]", annotatedseq)
    newseqs = []

    if matches:
        for mat in reversed(matches):
            if "Phospho" not in mat:
                annotatedseq = annotatedseq.replace(f"[{mat}]", "")
                matches.remove(mat)

        for i, p in enumerate(matches):
            newseq = annotatedseq.replace(f"[{p}]", "*", 1)
            newseq = newseq.replace(f"[{p}]", "")
            newseqs.append(newseq)
            annotatedseq = annotatedseq.replace(f'[{p}]', "", 1)
    return newseqs

def lower_site(seq):
    seq = seq.replace("_", "")
    star_index = seq.find('*') 
    if star_index != -1:
        seq = seq[:star_index -1] + seq[star_index-1].lower() + seq[star_index:]
    return seq
tri = phospho['Sequence'].apply(keep_phospho)
phospho['tempFixSeq'] = tri
phospho = phospho.explode('tempFixSeq')
phospho.reset_index(drop=True, inplace=True)
phospho.insert(phospho.columns.get_loc('Sequence') + 1, 'FixSeq', phospho.pop('tempFixSeq'))

# Explode the lists in 'FixSeq' into separate rows
phospho = phospho.explode('FixSeq')
# phospho = phospho.explode('TempFixSeq')
# phospho = phospho.explode('FixSeq')
# phospho.reset_index(drop= True, inplace=True)
# phospho.insert(phospho.columns.get_loc('Sequence'), "NewSequence", '')
# cdata['NewSequence'] = cdata['ModSequence']
# del cdata['ModSequence']
# cdata.reset_index(drop=True, inplace=True)
# cdata.insert(cdata.columns.get_loc('Sequence'), "MySequence", '')
phospho

,Accession,Description,Genes,Stripped_Seq,Sequence,FixSeq,Detected_Imputed,PeptidePosition,PTMLocations,112744 Fxn10_Control Normalized,...,Fxn5_Rxn_CV,Fxn6_Control_CV,Fxn6_Rxn_CV,Fxn7_Control_CV,Fxn7_Rxn_CV,Fxn8_Control_CV,Fxn8_Rxn_CV,Fxn9_Control_CV,Fxn9_Rxn_CV,SPQC_CV
0,A0A0B4J2A2,"Peptidyl-prolyl cis-trans isomerase A-like 4C,...",PPIAL4C,HTGSGILSMANAGPNTNGSQFFICTAK,_HTGSGILSMANAGPNT[Phospho (STY)]NGSQFFIC[Carba...,_HTGSGILSMANAGPNT*NGSQFFICTAK_,-.-.-.-.-.1.-.-.-.-.-.-.-.-.-.-.-.-.-.1.-.-.-....,92,"(T107,C115)",3.519489,...,66.90,48.80,90.7,38.9,58.3,86.9,81.1,119.00,117.0,46.3
1,A0A0B4J2D5;P0DPI2,Putative glutamine amidotransferase-like class...,GATD3B;GATD3,NLSTFAVDGKDCKVNK,_NLS[Phospho (STY)]T[Phospho (STY)]FAVDGKDC[Ca...,_NLS*TFAVDGKDCKVNK_,-.-.1.-.-.1.-.1.1.1.1.1.1.1.1.1.1.-.-.1.1.-.-....,142;142,"(S144,T145,C153);(S144,T145,C153)",5.129906,...,5.53,8.06,54.1,40.6,44.9,26.7,75.2,170.00,12.4,80.6
2,A0A0B4J2D5;P0DPI2,Putative glutamine amidotransferase-like class...,GATD3B;GATD3,NLSTFAVDGKDCKVNK,_NLS[Phospho (STY)]T[Phospho (STY)]FAVDGKDC[Ca...,_NLST*FAVDGKDCKVNK_,-.-.1.-.-.1.-.1.1.1.1.1.1.1.1.1.1.-.-.1.1.-.-....,142;142,"(S144,T145,C153);(S144,T145,C153)",5.129906,...,5.53,8.06,54.1,40.6,44.9,26.7,75.2,170.00,12.4,80.6
3,A0A0B4J2F2;P57059,Putative serine/threonine-protein kinase SIK1B...,SIK1B;SIK1,GGLSPFHAPAQSPGLHGGAAGSR,_GGLSPFHAPAQS[Phospho (STY)]PGLHGGAAGSR_,_GGLSPFHAPAQS*PGLHGGAAGSR_,-.-.-.1.1.1.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.1....,623;623,(S634);(S634),8.803518,...,86.30,99.30,59.0,53.2,146.0,37.3,67.7,57.90,18.6,131.0
4,A0A0B4J2F2;P57059,Putative serine/threonine-protein kinase SIK1B...,SIK1B;SIK1,GGLSPFHAPAQSPGLHGGAAGSR,_GGLS[Phospho (STY)]PFHAPAQSPGLHGGAAGSR_,_GGLS*PFHAPAQSPGLHGGAAGSR_,-.-.-.1.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-....,623;623,(S626);(S626),8.924532,...,53.50,56.50,41.8,91.8,63.4,38.2,34.7,6.79,12.7,45.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42481,Q9Y6Y8,"SEC23-interacting protein, org=Homo sapiens",SEC23IP,VVESPDFSKDEDYLGK,_VVES[Phospho (STY)]PDFSKDEDYLGK_,_VVES*PDFSKDEDYLGK_,1.1.-.1.1.2.1.1.-.1.1.1.1.2.-.1.2.-.2.1.1.2.-....,923,(S926),90.819400,...,23.90,27.40,24.4,39.9,36.7,49.3,18.9,34.30,22.3,25.1
42482,TRYP_PIG,"Trypsin, org=Sus scrofa",NaN,LSSPATLNSR,_LSSPATLNS[Phospho (STY)]R_,_LSSPATLNS*R_,1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1....,98,(S106),7.512516,...,46.90,32.40,31.2,34.9,85.8,81.0,30.3,161.00,152.0,78.2
42483,TRYP_PIG,"Trypsin, org=Sus scrofa",NaN,LSSPATLNSR,_LSSPAT[Phospho (STY)]LNSR_,_LSSPAT*LNSR_,-.1.-.-.1.-.1.1.-.1.1.1.-.1.1.-.1.1.-.1.1.1.1....,98,(T103),624.474396,...,40.60,45.10,66.9,87.8,54.2,31.6,97.7,47.30,85.7,46.2
42484,TRYP_PIG,"Trypsin, org=Sus scrofa",NaN,VATVSLPR,_VATVS[Phospho (STY)]LPR_,_VATVS*LPR_,-.1.-.-.1.-.1.-.-.1.-.-.-.1.-.1.1.1.1.1.1.-.-....,108,(S112),9.375021,...,48.50,29.60,29.4,50.5,32.2,50.6,24.2,47.80,61.1,62.0


In [19]:
import json 
if '~' not in phospho.loc[0, 'Accession']:
    phospho['Accession'] = phospho.groupby('Accession').cumcount().add(1).astype(str).radd('__').radd(phospho['Accession'])
    storeUniques = dict(zip(phospho['Accession'], phospho['FixSeq']))
    storeUniques
with open("../10415/phosphoUniqueSubstrates.json", "w") as outfile: 
    json.dump(storeUniques, outfile, indent=4)


In [20]:
phospho

,Accession,Description,Genes,Stripped_Seq,Sequence,FixSeq,Detected_Imputed,PeptidePosition,PTMLocations,112744 Fxn10_Control Normalized,...,Fxn5_Rxn_CV,Fxn6_Control_CV,Fxn6_Rxn_CV,Fxn7_Control_CV,Fxn7_Rxn_CV,Fxn8_Control_CV,Fxn8_Rxn_CV,Fxn9_Control_CV,Fxn9_Rxn_CV,SPQC_CV
0,A0A0B4J2A2__1,"Peptidyl-prolyl cis-trans isomerase A-like 4C,...",PPIAL4C,HTGSGILSMANAGPNTNGSQFFICTAK,_HTGSGILSMANAGPNT[Phospho (STY)]NGSQFFIC[Carba...,_HTGSGILSMANAGPNT*NGSQFFICTAK_,-.-.-.-.-.1.-.-.-.-.-.-.-.-.-.-.-.-.-.1.-.-.-....,92,"(T107,C115)",3.519489,...,66.90,48.80,90.7,38.9,58.3,86.9,81.1,119.00,117.0,46.3
1,A0A0B4J2D5;P0DPI2__1,Putative glutamine amidotransferase-like class...,GATD3B;GATD3,NLSTFAVDGKDCKVNK,_NLS[Phospho (STY)]T[Phospho (STY)]FAVDGKDC[Ca...,_NLS*TFAVDGKDCKVNK_,-.-.1.-.-.1.-.1.1.1.1.1.1.1.1.1.1.-.-.1.1.-.-....,142;142,"(S144,T145,C153);(S144,T145,C153)",5.129906,...,5.53,8.06,54.1,40.6,44.9,26.7,75.2,170.00,12.4,80.6
2,A0A0B4J2D5;P0DPI2__2,Putative glutamine amidotransferase-like class...,GATD3B;GATD3,NLSTFAVDGKDCKVNK,_NLS[Phospho (STY)]T[Phospho (STY)]FAVDGKDC[Ca...,_NLST*FAVDGKDCKVNK_,-.-.1.-.-.1.-.1.1.1.1.1.1.1.1.1.1.-.-.1.1.-.-....,142;142,"(S144,T145,C153);(S144,T145,C153)",5.129906,...,5.53,8.06,54.1,40.6,44.9,26.7,75.2,170.00,12.4,80.6
3,A0A0B4J2F2;P57059__1,Putative serine/threonine-protein kinase SIK1B...,SIK1B;SIK1,GGLSPFHAPAQSPGLHGGAAGSR,_GGLSPFHAPAQS[Phospho (STY)]PGLHGGAAGSR_,_GGLSPFHAPAQS*PGLHGGAAGSR_,-.-.-.1.1.1.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.1....,623;623,(S634);(S634),8.803518,...,86.30,99.30,59.0,53.2,146.0,37.3,67.7,57.90,18.6,131.0
4,A0A0B4J2F2;P57059__2,Putative serine/threonine-protein kinase SIK1B...,SIK1B;SIK1,GGLSPFHAPAQSPGLHGGAAGSR,_GGLS[Phospho (STY)]PFHAPAQSPGLHGGAAGSR_,_GGLS*PFHAPAQSPGLHGGAAGSR_,-.-.-.1.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-....,623;623,(S626);(S626),8.924532,...,53.50,56.50,41.8,91.8,63.4,38.2,34.7,6.79,12.7,45.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42481,Q9Y6Y8__4,"SEC23-interacting protein, org=Homo sapiens",SEC23IP,VVESPDFSKDEDYLGK,_VVES[Phospho (STY)]PDFSKDEDYLGK_,_VVES*PDFSKDEDYLGK_,1.1.-.1.1.2.1.1.-.1.1.1.1.2.-.1.2.-.2.1.1.2.-....,923,(S926),90.819400,...,23.90,27.40,24.4,39.9,36.7,49.3,18.9,34.30,22.3,25.1
42482,TRYP_PIG__1,"Trypsin, org=Sus scrofa",NaN,LSSPATLNSR,_LSSPATLNS[Phospho (STY)]R_,_LSSPATLNS*R_,1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1....,98,(S106),7.512516,...,46.90,32.40,31.2,34.9,85.8,81.0,30.3,161.00,152.0,78.2
42483,TRYP_PIG__2,"Trypsin, org=Sus scrofa",NaN,LSSPATLNSR,_LSSPAT[Phospho (STY)]LNSR_,_LSSPAT*LNSR_,-.1.-.-.1.-.1.1.-.1.1.1.-.1.1.-.1.1.-.1.1.1.1....,98,(T103),624.474396,...,40.60,45.10,66.9,87.8,54.2,31.6,97.7,47.30,85.7,46.2
42484,TRYP_PIG__3,"Trypsin, org=Sus scrofa",NaN,VATVSLPR,_VATVS[Phospho (STY)]LPR_,_VATVS*LPR_,-.1.-.-.1.-.1.-.-.1.-.-.-.1.-.1.1.1.1.1.1.-.-....,108,(S112),9.375021,...,48.50,29.60,29.4,50.5,32.2,50.6,24.2,47.80,61.1,62.0


In [21]:

all = pd.concat([kinases, phospho], axis=0)
print(f'Phospho shape: {phospho.shape}\nKinase shape {kinases.shape}\nConcatenatedFinal: {all.shape}')
all.to_csv(f'{os.path.dirname(AnnotatedKinaseSheet)}/KinSub{os.path.basename(AnnotatedKinaseSheet)}', index=False)



Phospho shape: (42486, 71)
Kinase shape (316, 53)
ConcatenatedFinal: (42802, 73)


In [12]:
all

,Accession,Human_Kinase,Description,Genes,Peptides,Detected_Imputed,112744 Fxn10_Control Normalized,112758 Fxn10_Control Normalized,112772 Fxn10_Control Normalized,112751 Fxn10_Rxn Normalized,...,Fxn5_Rxn_CV,Fxn6_Control_CV,Fxn6_Rxn_CV,Fxn7_Control_CV,Fxn7_Rxn_CV,Fxn8_Control_CV,Fxn8_Rxn_CV,Fxn9_Control_CV,Fxn9_Rxn_CV,SPQC_CV
0,A0A0B4J2F2_k,SIK1B_HUMAN,Putative serine/threonine-protein kinase SIK1B...,SIK1B;SIK1,15.0,10.12.10.10.11.13.11.5.9.10.7.7.9.5.6.10.9.4.5...,8966.244557,7308.774602,8763.373499,9420.749301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,O00141_k,SGK1_HUMAN,"Serine/threonine-protein kinase Sgk1, org=Homo...",SGK1,7.0,6.5.6.6.5.4.4.2.4.3.1.4.2.6.4.4.4.5.2.3.3.3.3....,15870.258819,10343.783089,11770.911566,12676.190954,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,O00238_k,BMR1B_HUMAN,"Bone morphogenetic protein receptor type-1B, o...",BMPR1B,1.0,-.-.-.-.-.-.1.-.-.-.-.-.1.-.1.1.1.1.-.-.-.-.-....,8.457218,13.655893,1.541628,14.093329,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,O00311_k,CDC7_HUMAN,"Cell division cycle 7-related protein kinase, ...",CDC7,30.0,26.25.24.23.25.20.13.9.13.8.10.13.13.16.12.14....,35625.073555,29760.044580,33417.690964,35242.215973,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,O00329_k,PK3CD_HUMAN,"Phosphatidylinositol 4,5-bisphosphate 3-kinase...",PIK3CD,13.0,8.7.6.10.11.8.7.3.5.3.6.6.6.5.7.10.5.6.6.7.6.9...,3381.358547,2288.085433,2117.901021,4732.677458,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42481,Q9Y6Y8~4,NaN,"SEC23-interacting protein, org=Homo sapiens",SEC23IP,NaN,1.1.-.1.1.2.1.1.-.1.1.1.1.2.-.1.2.-.2.1.1.2.-....,90.819400,114.393484,66.188372,235.425495,...,23.9,27.4,24.4,39.9,36.7,49.3,18.9,34.3,22.3,25.1
42482,TRYP_PIG~1,NaN,"Trypsin, org=Sus scrofa",NaN,NaN,1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1....,7.512516,5.905145,3.663518,1.021589,...,46.9,32.4,31.2,34.9,85.8,81.0,30.3,161.0,152.0,78.2
42483,TRYP_PIG~2,NaN,"Trypsin, org=Sus scrofa",NaN,NaN,-.1.-.-.1.-.1.1.-.1.1.1.-.1.1.-.1.1.-.1.1.1.1....,624.474396,896.785816,2216.152687,1.085434,...,40.6,45.1,66.9,87.8,54.2,31.6,97.7,47.3,85.7,46.2
42484,TRYP_PIG~3,NaN,"Trypsin, org=Sus scrofa",NaN,NaN,-.1.-.-.1.-.1.-.-.1.-.-.-.1.-.1.1.1.1.1.1.-.-....,9.375021,6.000127,3.592862,2.411671,...,48.5,29.6,29.4,50.5,32.2,50.6,24.2,47.8,61.1,62.0
